# Upload an image channel and its annotations to BossDB

This notebook uses the [intern](https://github.com/jhuapl-boss/intern) Python library to upload an image channel and its annotations to BossDB. The image channel is a 3D volume of 16-bit grayscale data. The annotations are a set of binary masks that are used to define the boundaries of the material types in the image channel. (In this volume, we have segmentation for `rock` and `bone`.)

We can use the Z-slice importer built into intern to convert image tiles into a 3D spatial database volume.

In [ ]:
# To point to the images on disk:
import pathlib

# To perform the upload:
from intern.convenience.array import ZSliceIngestJob

In [ ]:
# Here, we define the data products.
# The image channel is the only required field. (ANNOTATIONS = [] is valid.)


IMAGE =  {
    # The channel name to use on BossDB.org (i.e., 
    # bossdb://collection/experiment/channel).
    # For more details on the BossDB ontology, including how to choose a name 
    # for collections, experiments, and channels, see the documentation on the
    # bossdb website (https://bossdb.org/get-started) and the YouTube tutorial
    # at (https://youtu.be/gbbfWDThELU?t=82).
    "name": "during2022/paddlefish43um/image",

    # The path on disk to the image stack. This should be a folder that has a
    # stack of images in it.
    "path": pathlib.Path("data/Paddlefish 43.53 um/Paddlefish scanned at 43.53 um"),

    # The filename pattern to use to find the images. This is a glob, so "*" is
    # a wildcard. The glob will be applied to the path above, so if "path" is
    # "/data/images" and "pattern" is "*.png", then the glob will be applied to
    # "/data/images/*.png". 
    #
    # You can use pattern="*" to upload all files in the directory, or you can
    # use a more specific pattern to only upload certain files (e.g., "*.png").
    #
    # Image names should sort in the order that you want them to be uploaded.
    # GOOD: ["image01.png", "image02.png", "image03.png", ..., "image99.png"]
    #       Note the zero-padding to make sorting work.
    # BAD:  ["image1.png", "image2.png", "image3.png", ..., "image99.png"]
    #       This naming convention is bad because "image10.png" will sort 
    #       alphabetically before "image99.png".
    "pattern": "*.jp2",

    # The datatype of the image to use on BossDB.org. (For more information,
    # see the getting-started link above.)
    # Valid options for image data are:
    #   "uint8"
    #   "uint16"
    # Valid options for annotation data are:
    #   "uint32"
    #   "uint64"
    "dtype": "uint16",
}

ANNOTATIONS = [
    {
        "name": "during2022/paddlefish43um/bone",
        "path": pathlib.Path("data/Paddlefish 43.53 um/Bones 43.53 um"),
        "pattern": "*.jpg",
        "dtype": "uint64",
    },

    {
        "name": "during2022/paddlefish43um/rock",
        "path": pathlib.Path("data/Paddlefish 43.53 um/Rock 43.53 um"),
        "pattern": "*.jpg",
        "dtype": "uint64",
    },
]

## BossDB Authentication

There are two ways to authenticate an ingest with BossDB:
1. Use a credentials file (usually stored at `~/.intern/intern.cfg`)
2. Specify a token in code

### Credentials File

The credentials file is a simple text file that contains the following information:

```ini
[Default]
protocol = https
host = api.bossdb.io
token = <token>
```

To get your token, go to the [BossDB web portal](https://api.bossdb.io/), log in, and click on your username in the upper right corner. Then click on "API Token"... Or click [here](https://api.bossdb.io/v1/mgmt/token).

### Token in Code

If you don't want to use a credentials file, you can specify the token in code. This is useful if you are running this notebook on a remote server that doesn't have a credentials file.

However, you should treat your token like a password: Anyone with your token can impersonate you on BossDB services. For this reason, **it is important to NEVER commit your token to a public codebase or repository**. If you do accidentally share your token, you can revoke it by going to the [BossDB web portal](https://api.bossdb.io/), logging in, clicking on your username in the upper right corner, and clicking on "API Token", or by clicking [here](https://api.bossdb.io/v1/mgmt/token).

You can specify the same details as in the credentials file in code:

```python
boss_creds = {
    'protocol': 'https',
    'host': 'api.bossdb.io',
    'token': '<token>'
}
```

When you create a `ZSliceIngestJob` object, you can pass in the credentials dictionary by including it as a `boss_options` argument:

```python
job = ZSliceIngestJob(
    ...,
    boss_options=boss_creds
)
```

In [ ]:
# Create the ingest job. (This will not start the upload yet!)
job = ZSliceIngestJob(IMAGE, ANNOTATIONS)

In [ ]:
# Run the upload!
# This will show a progress bar for each of the channels (here, three total).
# You may see some warnings: as long as the cell completes and the upload() 
# call returns True, these are safe to ignore.
# (Sometimes intermittent network errors cause a chunk of the upload to fail,
# but the upload will automatically retry the failed chunks.)
job.upload()